#### Interpolation(보간법)

- 데이터의 결측치(NaN)나 중간값을 그럴듯하게 메워 데이터의 연속성과 패턴 유지
- 매우 중요한 전처리 기법
- 장점
    * 데이터 손실 최소화
    * 시간적 패턴 반영 및 유지
    * 시각적·해석적 일관성 유지

- DF/SR.interpolation()메서드
    * method 매개변수
        - 'linear'(기본), 'time', 'index', 'polynomial', 'spline',
        'pad'(=ffill), 'nearest' 등
    * axis 매개변수
        - 0(열별, 위->아래) / 1(행별, 왼->오)
    * limit, limit_direction 매개변수
        - 긴 결측 구간에서 메울 개수/방향 제어
    * 비수치형 컬럼은 자동 제외

In [1]:
import numpy as np      ## 결측치
import pandas as pd     ## 데이터 분석용 패키지

In [2]:
## 테스트용 데이터
s = pd.Series([1.0, np.nan, np.nan, 4.0, 5.0])
print('원본:\n', s)

## ----------------------------------------------------
## 기본(method='linear')은 인덱스 순서 기준 직선 보간
## ----------------------------------------------------
s_lin = s.interpolate()
print("\nlinear 보간:\n", s_lin)

원본:
 0    1.0
1    NaN
2    NaN
3    4.0
4    5.0
dtype: float64

linear 보간:
 0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64


In [3]:
##
## 인덱스 간격을 고려한 보간 : method='index'
##
s2 = pd.Series( [1.0, np.nan,  4.0, np.nan],
               index=[0,10,20,30])  # 인덱스 간격이 불균일

print('원본:\n', s2)
print('\nlinear(기본):\n', s2.interpolate())                # 위치(순서) 기준
print('\nnindex 기준:\n', s2.interpolate(method='index'))   # 인덱스 값 거리


원본:
 0     1.0
10    NaN
20    4.0
30    NaN
dtype: float64

linear(기본):
 0     1.0
10    2.5
20    4.0
30    4.0
dtype: float64

nindex 기준:
 0     1.0
10    2.5
20    4.0
30    4.0
dtype: float64


In [4]:
## -------------------------------------------------
## 시계열 보간
## -------------------------------------------------
date_idx = pd.to_datetime([
    "2025-01-01", "2025-01-02", "2025-01-03", "2025-01-06", "2025-01-07"
])
ts = pd.Series([10, np.nan, np.nan, 40, 50], index=date_idx)

print('원본\n', ts)
print('\n시간기반 보간(time):\n', ts.interpolate(method='time'))

원본
 2025-01-01    10.0
2025-01-02     NaN
2025-01-03     NaN
2025-01-06    40.0
2025-01-07    50.0
dtype: float64

시간기반 보간(time):
 2025-01-01    10.0
2025-01-02    16.0
2025-01-03    22.0
2025-01-06    40.0
2025-01-07    50.0
dtype: float64


In [5]:
## 다항식/ 스플라인 보간(연속성/곡률 보간)
## 다항식 보간: order 필요(scipy없이 가능)
s3=pd.Series([0,np.nan,1,np.nan,4],index=[0,1,2,3,4])

print("polynomial(order=2):\n",s3.interpolate(method='polynomial',order=2))

## 스플라인 보간: Scipy 필요 (설치되어 있어야함)
## pip install scipy
# s3.interpolate(method='spline',order=2)

polynomial(order=2):
 0    0.00
1    0.25
2    1.00
3    2.25
4    4.00
dtype: float64


In [6]:
## --------------------------------------------
## DataFrame 전체 보간 + 축 지정
## --------------------------------------------
df = pd.DataFrame({
    "A":[1,np.nan,3,np.nan,5],
    "B":[np.nan,2,np.nan,4,np.nan]
})

print("원본:\n", df)

print("\n열별 보간(axis=0)\n", df.interpolate())
print("\n컬럼별 보간(axis=1)\n", df.interpolate(axis=1))

원본:
      A    B
0  1.0  NaN
1  NaN  2.0
2  3.0  NaN
3  NaN  4.0
4  5.0  NaN

열별 보간(axis=0)
      A    B
0  1.0  NaN
1  2.0  2.0
2  3.0  3.0
3  4.0  4.0
4  5.0  4.0

컬럼별 보간(axis=1)
      A    B
0  1.0  1.0
1  NaN  2.0
2  3.0  3.0
3  NaN  4.0
4  5.0  5.0


In [7]:
## -------------------------------------------------
## 결측 연속 구간 길이 제한 : limit / limit_direction
## -------------------------------------------------
s4 = pd.Series([1, np.nan, np.nan, np.nan, 5])

print(s4.interpolate(limit=1, limit_direction='forward'))
print(s4.interpolate(limit=2, limit_direction='both'))

0    1.0
1    2.0
2    NaN
3    NaN
4    5.0
dtype: float64
0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64


In [9]:
##
## 끝단(양끝) 결측 처리 주의점 & 팁
##
s5 = pd.Series([np.nan, 2, np.nan, 4, np.nan])

# 보간은 내부 구간을 메우는 데 강함. 양끝은 남을 수 있음
print("기본 보간:\n", s5.interpolate())

# 보간 전/후로 ffill/bfill을 조합하기도 함
filled = s5.interpolate().ffill().bfill()
print('\n보간 + ffill/bfill:\n', filled)

기본 보간:
 0    NaN
1    2.0
2    3.0
3    4.0
4    4.0
dtype: float64

보간 + ffill/bfill:
 0    2.0
1    2.0
2    3.0
3    4.0
4    4.0
dtype: float64
